In [8]:
import os
from dotenv import load_dotenv

load_dotenv()
VANTAGE_KEY = os.environ['VANTAGE_API']
assert VANTAGE_KEY is not None

In [9]:
import polars as pl
sp500_table = pl.read_csv('data/sp500.csv').with_row_index('company_id')
test_symbols = list(sp500_table.sample(5)['Symbol'])

In [10]:
import boto3
import json

In [11]:
import requests
# income_data = []
# for sym in test_symbols:
#     url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={sym}&apikey={VANTAGE_KEY}'
#     r = requests.get(url)
#     income_data.append(r.json())

# balance_data = []
# # for sym in test_symbols:
# url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol=AAP&apikey={VANTAGE_KEY}'
# r = requests.get(url)
# balance_data.append(r.json())

# response.json()

In [5]:
import requests
url = 'https://www.alphavantage.co/query'
params = {
        'function': 'INCOME_STATEMENT',
        'symbol': 'AAP',
        'apikey': VANTAGE_KEY
        }
# response = requests.get(url, params=params)
from datetime import datetime
datetime.today().strftime('%Y-%m')

'2024-10'

In [14]:
balance_list = []

# Loop through each company's API response
for response in balance_data:
    # Extract annualReports data
    df = pl.DataFrame(response['annualReports'])
    
    # Add the company symbol as a new column
    df = df.with_columns(
        pl.lit(response['symbol']).alias("company_id"),   
    )
    
    # Append the DataFrame to the list
    balance_list.append(df)

# Combine all DataFrames into one
balance_df = pl.concat(balance_list)

In [10]:
income_list = []

# Loop through each company's API response
for response in income_data:
    # Extract annualReports data
    df = pl.DataFrame(response['annualReports'])
    
    # Add the company symbol as a new column
    df = df.with_columns(
        pl.lit(response['symbol']).alias("company_id"),   
    )
    
    # Append the DataFrame to the list
    income_list.append(df)

# Combine all DataFrames into one
income_df = pl.concat(income_list)


In [6]:
import duckdb
import os
AWS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY = os.environ['AWS_SECRET_ACCESS_KEY']
# Configure S3 access
duckdb.sql(f"""
    INSTALL httpfs;
    LOAD httpfs;
    SET s3_region='us-east-1'; -- Set your region
    SET s3_access_key_id={AWS_KEY};
    SET s3_secret_access_key={AWS_SECRET_KEY};
""")


/var/folders/x6/5963y0d96z7dql2l8nq8v91m0000gn/T/ipykernel_50556/546509196.py:1: UserWarning: '(default_)region' not set; polars will try to get it from bucket

Set the region manually to silence this warning.
  combined_data = pl.read_parquet('s3://mote-finder/stage/combined_BALANCE.parquet')


In [70]:
# final_df = df.with_columns(
#     pl.col('fiscalDateEnding').str.to_date('%Y-%m-%d'),
#     pl.col("*").exclude('fiscalDateEnding', 'reportedCurrency', 'company_id').cast(pl.Float64, strict=False)
# )
final_df.with_columns(
    (pl.col('incomeTaxExpense')/pl.col('incomeBeforeTax')).alias('taxRate')
).with_columns(
    (pl.col('operatingIncome')*(1-pl.col('taxRate'))).alias('NOPAT')
)

fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,taxRate,NOPAT
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-12-31,"""USD""",4.5235e9,1.1284e10,1.1203e10,6.7641e9,1.14377e8,4.4091e9,null,4.4386e9,null,-8.5939e7,null,8.8055e7,1.1284e10,5.525e6,2.769e8,2.95e7,3.1847e7,2.112e6,0.0,2.9735e7,2.2198e7,1.14377e8,1.43877e8,2.9735e7,0.066317,1.0679e8
2022-12-31,"""USD""",4.9322e9,1.1143e10,1.0472e10,6.1926e9,6.70253e8,4.2620e9,null,4.2789e9,null,-5.106e7,5.546e7,5.106e7,1.1143e10,-6.996e6,2.528e8,3.1e7,6.48687e8,1.3996e8,7.408e6,5.01872e8,4.79356e8,7.14151e8,7.45151e8,4.64402e8,0.215759,5.2564e8
2021-12-31,"""USD""",4.9287e9,1.0996e10,1.0190e10,6.0692e9,8.38717e8,4.0900e9,null,4.1211e9,null,-3.7791e7,3.7791e7,3.7791e7,1.0996e10,4.999e6,2.288e8,3.11e7,8.05925e8,1.89817e8,0.0,6.16108e8,6.2024e8,8.38717e8,8.69817e8,6.16108e8,0.235527,6.4118e8
2020-12-31,"""USD""",4.4816e9,1.0051e10,9.3882e9,5.6247e9,7.49907e8,3.7317e9,null,3.7635e9,null,-4.6886e7,4.6886e7,4.6886e7,1.0051e10,-3.984e6,2.185e8,3.16e7,6.51015e8,1.57994e8,4.8022e7,4.93021e8,5.00831e8,7.49907e8,7.81507e8,4.93021e8,0.242689,5.6791e8
2019-12-31,"""USD""",4.2547e9,9.7073e9,9.0658e9,5.4543e9,6.7718e8,3.5776e9,null,3.6116e9,null,-3.9898e7,4.1598e7,3.9898e7,9.7073e9,1.122e7,2.067e8,3.17e7,6.37746e8,1.5085e8,1.0756e7,4.86896e8,4.9652e8,6.7718e8,7.0888e8,4.86896e8,0.236536,5.1700e8
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2014-12-31,"""USD""",4.4536e9,9.7705e9,9.0867e9,5.3902e9,8.5171e8,3.6019e9,null,3.6964e9,null,-7.3408e7,null,7.3408e7,null,3.092e6,2.3504e8,5.6499e7,7.81394e8,2.87569e8,7.3408e7,4.93825e8,4.77805e8,8.54802e8,9.11301e8,4.93825e8,0.36802,5.3826e8
2013-12-31,"""USD""",3.2521e9,6.4496e9,5.8255e9,3.2417e9,6.60318e8,2.5918e9,null,2.6221e9,null,-3.6618e7,null,3.6618e7,-7.582e6,2.698e6,1.99821e8,-7.974e6,6.26398e8,2.3464e8,3.6618e7,3.91758e8,7.84532e8,6.63016e8,6.55042e8,3.91758e8,0.374586,4.1297e8
2012-12-31,"""USD""",3.0980e9,6.1639e9,5.5458e9,3.1070e9,6.57315e8,2.4407e9,null,2.4754e9,null,-3.3841e7,null,3.3841e7,-7.299e6,600000.0,1.85909e8,-3.635e6,6.24074e8,2.36404e8,3.3841e7,3.8767e8,7.75203e8,6.57915e8,6.5428e8,3.8767e8,0.378808,4.0832e8


In [31]:
# %%timeit
# import json
# import s3fs
# # fs = s3fs.S3FileSystem()
# with fs.open(income_statement_path, 'rb') as f:
#     s3_clientdata = json.load(f)
# df = pl.DataFrame(s3_clientdata['annualReports'])

In [222]:

cdf = pl.concat([df, df])
cdf.unique(subset=['compa'])

fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2023-12-31""","""USD""","""4523502000""","""11284207000""","""11202730000""","""6764105000""","""114377000""","""4409125000""","""None""","""4438625000""","""None""","""-85939000""","""None""","""88055000""","""11284207000""","""5525000""","""276900000""","""29500000""","""31847000""","""2112000""","""0""","""29735000""","""22198000""","""114377000""","""143877000""","""29735000"""
"""2022-12-31""","""USD""","""4932235000""","""11142914000""","""10471571000""","""6192622000""","""670253000""","""4261982000""","""None""","""4278949000""","""None""","""-51060000""","""55460000""","""51060000""","""11142914000""","""-6996000""","""252800000""","""31000000""","""648687000""","""139960000""","""7408000""","""501872000""","""479356000""","""714151000""","""745151000""","""464402000"""
"""2021-12-31""","""USD""","""4928748000""","""10996289000""","""10190372000""","""6069241000""","""838717000""","""4090031000""","""None""","""4121131000""","""None""","""-37791000""","""37791000""","""37791000""","""10996289000""","""4999000""","""228800000""","""31100000""","""805925000""","""189817000""","""0""","""616108000""","""620240000""","""838717000""","""869817000""","""616108000"""
"""2020-12-31""","""USD""","""4481614000""","""10051399000""","""9388214000""","""5624707000""","""749907000""","""3731707000""","""None""","""3763507000""","""None""","""-46886000""","""46886000""","""46886000""","""10051399000""","""-3984000""","""218500000""","""31600000""","""651015000""","""157994000""","""48022000""","""493021000""","""500831000""","""749907000""","""781507000""","""493021000"""
"""2019-12-31""","""USD""","""4254746000""","""9707303000""","""9065823000""","""5454257000""","""677180000""","""3577566000""","""None""","""3611566000""","""None""","""-39898000""","""41598000""","""39898000""","""9707303000""","""11220000""","""206700000""","""31700000""","""637746000""","""150850000""","""10756000""","""486896000""","""496520000""","""677180000""","""708880000""","""486896000"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2014-12-31""","""USD""","""4453613000""","""9770453000""","""9086698000""","""5390248000""","""851710000""","""3601903000""","""None""","""3696450000""","""None""","""-73408000""","""None""","""73408000""","""None""","""3092000""","""235040000""","""56499000""","""781394000""","""287569000""","""73408000""","""493825000""","""477805000""","""854802000""","""911301000""","""493825000"""
"""2013-12-31""","""USD""","""3252146000""","""6449614000""","""5825522000""","""3241668000""","""660318000""","""2591828000""","""None""","""2622071000""","""None""","""-36618000""","""None""","""36618000""","""-7582000""","""2698000""","""199821000""","""-7974000""","""626398000""","""234640000""","""36618000""","""391758000""","""784532000""","""663016000""","""655042000""","""391758000"""
"""2012-12-31""","""USD""","""3098036000""","""6163863000""","""5545814000""","""3106967000""","""657315000""","""2440721000""","""None""","""2475387000""","""None""","""-33841000""","""None""","""33841000""","""-7299000""","""600000""","""185909000""","""-3635000""","""624074000""","""236404000""","""33841000""","""387670000""","""775203000""","""657915000""","""654280000""","""387670000"""


In [125]:
BUCKET_NAME='mote-finder'
prefix = 'raw/'
def list_json_files(prefix):
    response = s3_obj.list_objects_v2(Bucket=BUCKET_NAME, Prefix=prefix)
    files = [content['Key'] for content in response.get('Contents', []) if content['Key'].endswith('.json')]
    return files

In [135]:
# response = s3_obj.list_objects_v2(Bucket='mote-finder', Prefix='raw/')
# files = [content['Key'] for content in response.get('Contents', []) if content['Key'].endswith('.json')]
# files = list_json_files('raw/')


In [29]:
# import os
# income_files = [file for file in files if 'INCOME' in file]
# balance_files = [file for file in files if 'BALANCE' in file]
# def save_to_s3_as_parquet(dataframe, existing_file_path):
#     """
#     Save the Polars dataframe as a Parquet file into S3.
#     """
#     if os.path.exists(existing_file_path):
#         # Load the existing data from Parquet
#         existing_data = pl.read_parquet(existing_file_path)

#         # Append the new data to the existing data
#         combined_data = pl.concat([existing_data, dataframe], rechunk=True)

#         # Write back the combined data to the Parquet file (overwrite)
#         combined_data.write_parquet(existing_file_path)
#     else:
#         # If the file doesn't exist, just write the new data
#         dataframe.write_parquet(existing_file_path)

#     s3_obj.upload_file(existing_file_path, BUCKET_NAME)
    
# combined_data = pl.DataFrame()

# for file in:

#     json_file = s3_obj.get_object(Bucket=BUCKET_NAME, Key=file)
#     # print(json_file)
#     data = json.loads(json_file['Body'].read())
#     print(data)
    # Focus on annualReports and convert to Polars DataFrame
    # reports_df = pl.DataFrame(data['annualReports'])

    # # Add company symbol as a column
    # reports_df = reports_df.with_columns([
    #     pl.lit(data['symbol']).alias("company_id")
    # ]).with_columns(
    #     pl.col('fiscalDateEnding').str.to_date('%Y-%m-%d'),
    #     pl.col("*").exclude('fiscalDateEnding', 'reportedCurrency', 'company_id').cast(pl.Float64, strict=False)
    # )
    # # Combine into a master dataframe
    # combined_data = pl.concat([combined_data, reports_df], rechunk=True)

# Save the combined data back into S3 or a database
# For this, you can store as Parquet, CSV, or another format
# save_to_s3_as_parquet(combined_data, f'stage/INCOME_STATEMENT_combined.parquet')





In [154]:
transfored_companies = set(combined_data['company_id'].unique().to_list())

In [152]:
company_names = set([f.split('/')[1] for f in income_files])

In [177]:
missing_files = list(company_names.difference(transfored_companies))
# to_fix_companies = []
for mc in missing_files:
    files = list_json_files(f'raw/{mc}/')
    for f in files:
        json_file = s3_obj.get_object(Bucket=BUCKET_NAME, Key=f)
        # print(json_file)
        data = json.loads(json_file['Body'].read())
        if 'annualReports' in data:
            print(mc)
            print("company contains info")
            print(data)
    # to_fix_companies.append(mc)

In [96]:
S3_CLIENT = boto3.client('s3')
def list_s3_files(s3_prefix):
    # s3 = boto3.client('s3')
    response = S3_CLIENT.list_objects_v2(Bucket='mote-finder', Prefix=s3_prefix)
    # print(response)
    return [content['Key'] for content in response.get('Contents', [])]

s3_keys = list_s3_files('raw/')
# parquet_paths = list_s3_files("stage/")
endpoint_keys = [file for file in s3_keys if 'EARNINGS' in file]
endpoint_keys
for s3_key in endpoint_keys:
    # print(s3_key)
    response = S3_CLIENT.get_object(Bucket='mote-finder', Key=s3_key)
    # print(response[''])
    json_data = json.loads(response['Body'].read())
    # Process and insert into table
    reports_df = pl.DataFrame(json_data['annualEarnings'])
# json_data




['stage/combined_BALANCE.parquet', 'stage/combined_INCOME.parquet']

In [105]:
fs = list_s3_files('stage/combined_BALANCE')
fs[0]

'stage/combined_BALANCE.parquet'

### Building the Big 5

In [32]:
# additional transformations
balance_df = pl.read_parquet('s3://mote-finder/stage/combined_BALANCE.parquet')
income_df = pl.read_parquet('s3://mote-finder/stage/combined_INCOME.parquet')


In [73]:
full_df = income_df.join(balance_df, how = 'inner', on = ['company_id', 'fiscalDateEnding'])
full_df = full_df.with_columns(
    (pl.col('incomeTaxExpense')/pl.col('incomeBeforeTax')).alias('taxRate')
).with_columns(
    (pl.col('operatingIncome')*(1-pl.col('taxRate'))).alias('NOPAT'),
    (pl.col('totalAssets') - pl.col('totalLiabilities')).alias('Equity')
    
).with_columns(
    (pl.col('NOPAT')/(pl.col('Equity') + pl.col('longTermDebt'))).alias('ROIC')
)
roic_df = full_df.select(['company_id', 'fiscalDateEnding',  'ROIC'])

In [ ]:
equity, EPS, Sales, cash growth rate

In [108]:
roic_df.sort('company_id')

company_id,fiscalDateEnding,ROIC
str,date,f64
"""AAP""",2023-12-31,0.0248
"""AAP""",2022-12-31,0.138784
"""AAP""",2019-12-31,0.120334
"""AAP""",2021-12-31,0.154032
"""AAP""",2020-12-31,0.123661
…,…,…
"""MMM""",2020-12-31,0.181312
"""MMM""",2009-12-31,0.177338
"""MMM""",2012-12-31,0.191493
